In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy

%matplotlib inline

In [2]:
import xml.etree.ElementTree as ET


def get_diagnosis(fillename: str) -> None:
    root = ET.parse(filename).getroot()
    # node = root
    # for i, e in enumerate(node):
    #     print(i, "\t", e)

    print("Diagnosis:", "\n")

    node = root[6]

    for i in node:
        if i.tag == "DiagnosisStatement":
            for j in i:
                if j.tag == "StmtText":
                    print(j.text)

In [3]:
def plot_ecg(X):
    leads = ["I", "II", "III", "aVR", "aVL", "aVF", "V1", "V2", "V3", "V4", "V5", "V6"]
    for c in range(12):
        plt.figure(figsize=(20, 5))
        plt.plot(X[:, c])
        plt.title(leads[c])
        plt.grid()

    plt.show()

In [4]:
data = [
    pd.read_csv(f"data/patient_{i}.csv").drop(columns="Unnamed: 12") for i in [1, 2]
]

In [5]:
X = scipy.signal.resample(data, 4000, axis=-2)[...]
X = np.pad(X, ((0, 0), (48, 48), (0, 0)), "constant")
X = X / 100
print(X.shape)

(2, 4096, 12)


In [6]:
# plot_ecg(X[0])

In [7]:
import tensorflow as tf
from keras.models import load_model
from keras.optimizers import Adam

tf.compat.v1.disable_eager_execution()
model = load_model("model/model.hdf5", compile=False)
model.compile(loss="binary_crossentropy", optimizer=Adam())

2023-01-12 16:57:48.961637: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Colocations handled automatically by placer.


2023-01-12 16:57:52.546362: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-12 16:57:52.548314: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-01-12 16:57:52.567124: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [8]:
Y = model.predict(X)
print(Y)
print(Y > 0.5)

/users/eleves-b/2020/nhat.vo/.conda/envs/thesis/lib/python3.9/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[[5.0600321e-04 2.9564858e-04 8.2674196e-05 9.6254404e-08 6.3298506e-01
  1.3814695e-04]
 [6.0870657e-06 1.3384314e-07 4.8042050e-09 9.6219708e-08 1.5358246e-08
  3.4617647e-06]]
[[False False False False  True False]
 [False False False False False False]]


In [9]:
import innvestigate

In [17]:
smoothgrad = innvestigate.create_analyzer("gradient", model)
gradient_analyzer = innvestigate.analyzer.Gradient(model)

In [ ]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 signal (InputLayer)            [(None, 4096, 12)]   0           []                               
                                                                                                  
 conv1d_1 (Conv1D)              (None, 4096, 64)     12288       ['signal[0][0]']                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 4096, 64)    256         ['conv1d_1[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 activation_1 (Activation)      (None, 4096, 64)     0           ['batch_normalization_1[0][

In [20]:
analysis = gradient_analyzer.analyze(X[0])

InvalidArgumentError: You must feed a value for placeholder tensor 'signal' with dtype float and shape [?,4096,12]
	 [[{{node signal}}]]

In [13]:
innvestigate.__version__

'2.0.0'